# Import Packages

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import fetch_california_housing
import math
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer

import optuna
from optuna.samplers import TPESampler

In [ ]:
train_set = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\train.csv")
test_set = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\test.csv")
sample_sub = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\sample_submission.csv")

In [ ]:
columns_to_vectorize = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for vector_target in columns_to_vectorize:
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_set[vector_target])
    train_set[f'{vector_target}_v'] = vectorizer.transform(train_set[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_set[vector_target])
    test_set[f'{vector_target}_v'] = vectorizer.transform(test_set[vector_target]).toarray().argmax(axis=1)[:,None]

In [ ]:
train_set

In [ ]:
test_set

In [ ]:
train_set.columns

In [ ]:
print('train_set shape: ', train_set.shape)
print('test_set shape: ', test_set.shape)

In [ ]:
train_set.info()

In [ ]:
train_set.describe()

## Missing Data

In [ ]:
total = train_set.isnull().sum().sort_values(ascending=False)
percent = (train_set.isnull().sum()/train_set.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

## Data visualisation

In [ ]:
features = ['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'gender_v', 'ever_married_v', 'work_type_v', 'Residence_type_v', 'smoking_status_v']
target = ['stroke']

In [ ]:
#https://github.com/dhaitz/mplcyberpunk
%pip install mplcyberpunk
import mplcyberpunk

In [ ]:
from itertools import islice, cycle

def add_secondary_plot(df, column, target_column, ax, n_bins, color=3, show_yticks=False, marker="."):
    secondary_ax = ax.twinx()
    bins = pd.cut(df[column], bins=n_bins)
    bins = pd.IntervalIndex(bins)
    bins = (bins.left + bins.right) / 2
    target = df.groupby(bins)[target_column].mean()
    target.plot(
        ax=secondary_ax, linestyle='',
        marker=marker, color=color, label=f"Mean '{target_column}'"
    )
    secondary_ax.grid(visible=False)
    
    if not show_yticks:
        secondary_ax.get_yaxis().set_ticks([])
        
    return secondary_ax

def render_feature_distros(train_df, test_df, features=[], labels=[], n_bins=50, n_cols=4, pad=2, h_pad=4, w_pad=None):
    histplot_hyperparams = {
        'kde':True,
        'alpha':0.4,
        'stat':'percent',
        'bins':n_bins
    }
    markers = ['.', '+', 'x', '1', '2']
    
    n_rows = math.ceil(len(features) / n_cols)
    cell_with_dim = 4
    cell_height_dim = 3
    
    fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * cell_with_dim, n_rows * cell_height_dim))
    plt.tight_layout(pad=pad, h_pad=h_pad, w_pad=w_pad, rect=None)
    plt.style.use("cyberpunk")
    
    # delete exess subplots
    for a in ax[n_rows - 1, int(((n_rows - (len(features) / n_cols)) * n_cols*-1)):]:
        a.axis('off')
        
    leg_handles = []
    leg_labels = []
    
    axs = []

    for i, feature in enumerate(features):
        row = math.ceil(i / n_cols) - 1
        col = (i % n_cols)
        sns.histplot(train_df[feature], label='Train X', ax=ax[row, col], **histplot_hyperparams)
        sns.histplot(test_df[feature], label='Test X', ax=ax[row, col], **histplot_hyperparams)
        ax[row, col].set_title(f'{feature} Distribution')
        mplcyberpunk.make_lines_glow(ax[row, col])
        axs.append(ax[row, col].get_legend_handles_labels())
        starting_at_three = islice(mplcyberpunk.cyberpunk_stylesheets['cyberpunk']['axes.prop_cycle'], 2, None)
        for j, label in enumerate(labels):
            sub_ax = add_secondary_plot(train_df, feature, label, ax[row, col], n_bins, color=next(starting_at_three)['color'], marker=markers[j])
            axs.append(sub_ax.get_legend_handles_labels())
        
    for axis in axs:
        if axis[1][0] not in leg_labels:
            leg_labels.extend(axis[1])
            leg_handles.extend(axis[0])
        
    fig.legend(leg_handles, leg_labels, loc='upper center', bbox_to_anchor=(0.5, 1.04), fontsize=14, ncol=len(features) + 2)

In [ ]:
render_feature_distros(train_df=train_set, test_df=test_set, features=features, labels=['stroke'])

### Correlation matrix (heatmap style)

In [ ]:
corrmat = train_set[features+target].corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, square=True, annot=True, fmt='.2f', cmap='seismic', vmin=-1, vmax=1)

# Train Model

In [ ]:
import lightgbm as lgbm
from xgboost import XGBRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import roc_curve, auc,recall_score,precision_score

In [ ]:
scaler = MinMaxScaler().fit(train_set[features])
X = scaler.transform(train_set[features])
X_test = scaler.transform(test_set[features])

scaler = MinMaxScaler().fit(train_set[target])
Y = scaler.transform(train_set[target])

In [ ]:
kf = KFold(n_splits=10, random_state=1, shuffle=True)
clfs = []
err = []

In [ ]:
y_min = Y.min()
y_max = Y.max()

print(y_min, y_max)

def my_rmse(y_true, y_hat):
    y_true[y_true < y_min] = y_min
    y_true[y_true > y_max] = y_max
    
    y_hat[y_hat < y_min] = y_min
    y_hat[y_hat > y_max] = y_max
    
    y_true_nan = np.isnan(y_true)
    y_hat_nan = np.isnan(y_hat)
    
    if y_true_nan.sum() > 0:
        print(y_true_nan.sum())
        np.where(y_true_nan, np.ma.array(y_true, mask=np.isnan(y_true)).mean(axis=0), y_true)
    if y_hat_nan.sum() > 0:
        print(y_hat_nan.sum())
        np.where(y_hat_nan, np.ma.array(y_hat, mask=np.isnan(y_hat)).mean(axis=0), y_hat)
    
    return mean_squared_error(y_true, y_hat, squared=False)

In [ ]:
def xgb_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

    param_grid = {
        'max_depth': trial.suggest_int('max_depth', 4, 20), # Extremely prone to overfitting!
        'n_estimators': trial.suggest_int('n_estimators', 2, 100, 1), # Extremely prone to overfitting!
        'eta': trial.suggest_float('eta', 0.0007, 0.113), # Most important parameter.
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 40), # L2 regularization

    } 
    
    reg = xgb.XGBModel(
        # These parameters should help with trial speed.
        objective='binary:logistic',
        tree_method='gpu_hist',
        booster='gbtree',
        predictor='gpu_predictor',
        n_jobs=4,
        eval_metric='auc',
        **param_grid
    )
    
    reg.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            verbose=False)
    
    preds = reg.predict(X_valid)
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    xgb_ranks[roc_auc] = reg
    
    # Returns the best RMSE for the trial.
    # Readers may want to try returning a cross validation score here.
    print(roc_auc)
    return roc_auc

In [ ]:
xgb_ranks = {}

train_time = 1 * 60 * 60
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='XGBRegressor')
study.optimize(xgb_objective, timeout=train_time)


In [ ]:
trial = study.best_trial

In [ ]:
trial.params

In [ ]:
len(xgb_ranks.keys())

In [ ]:
xgb_ranks.keys()

In [ ]:
top_50 = sorted(list(xgb_ranks.keys()))[-100:]

In [ ]:
me_preds = []
for key in top_50:
    me_preds.append(xgb_ranks[key].predict(X_test))

final_preds = np.stack(me_preds).mean(0)

In [ ]:
len(final_preds)

In [ ]:
XGB_submission = pd.DataFrame(data={'id': test_set.id, 'stroke': final_preds})
XGB_submission.to_csv(fr'D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\XGB.csv', index=False)

In [ ]:
me_preds = []
for key in top_50:
    me_preds.append(xgb_ranks[key].predict(X))

final_preds = np.stack(me_preds).mean(0)
train_set['predicted_XGB_stroke'] = final_preds

In [ ]:
render_feature_distros(train_df=train_set, test_df=test_set, features=features, labels=['stroke', 'predicted_XGB_stroke'])

In [ ]:
def lgbm_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 100, 1000), 
        'num_leaves': trial.suggest_int('num_leaves', 100, 10000),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10), 
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000), 
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 100), 
        'n_estimators': trial.suggest_int('n_estimators', 10, 100000), 
    } 
    

    clf = lgbm.LGBMRegressor(**param_grid,
                             metric='AUC',
                             random_state=1)
    
    clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)],
            verbose=False)
    preds = clf.predict(X_valid)
    
    #rmse = mean_squared_error(y_val, preds, squared=False)
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    lgbm_ranks[roc_auc] = clf
    
    # Returns the best RMSE for the trial.
    # Readers may want to try returning a cross validation score here.
    print(roc_auc)
    return roc_auc

In [ ]:
lgbm_ranks = {}

train_time = 1 * 60 * 60
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='XGBRegressor')
study.optimize(lgbm_objective, timeout=train_time)



In [ ]:
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial


print('\tValue: {}'.format(trial.value))
print('\tParams: ')
for key, value in trial.params.items():
    print('\t\t{}: {}'.format(key, value))

In [ ]:
len(lgbm_ranks.keys())

In [ ]:
top_50 = sorted(list(lgbm_ranks.keys()))[-100:]
top_50

In [ ]:
me_preds = []
for key in top_50:
    me_preds.append(lgbm_ranks[key].predict(X_test))

final_preds = np.stack(me_preds).mean(0)

In [ ]:
LGBM_submission = pd.DataFrame(data={'id': test_set.id, 'stroke': final_preds})
LGBM_submission.to_csv(fr'D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\LGBM.csv', index=False)

In [ ]:
me_preds = []
for key in top_50:
    me_preds.append(lgbm_ranks[key].predict(X))

final_preds = np.stack(me_preds).mean(0)

In [ ]:
train_set['predicted_LGBM_stroke'] = final_preds

In [ ]:
import math

n_bins = 50
histplot_hyperparams = {
    'kde':True,
    'alpha':0.4,
    'stat':'percent',
    'bins':n_bins
}

n_cols = 4
n_rows = math.ceil(len(features) / n_cols)
cell_with_dim = 4
cell_height_dim = 3

fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * cell_with_dim, n_rows * cell_height_dim))

plt.tight_layout(pad=2, h_pad=4, w_pad=None, rect=None)
plt.style.use("cyberpunk")

for a in ax[n_rows - 1, int(((n_rows - (len(features) / n_cols)) * n_cols*-1)):]:
    a.axis('off')

handles = []
labels = []
    
for i, feature in enumerate(features):
    row = math.ceil(i / n_cols) - 1
    col = (i % n_cols)
    sns.histplot(train_set[feature], label='Train X', ax=ax[row, col], **histplot_hyperparams)
    sns.histplot(test_set[feature], label='Test X', ax=ax[row, col], **histplot_hyperparams)
    ax[row, col].set_title(f'{feature} Distribution');
    ax2 = add_secondary_plot(train_set, feature, target[0], ax[row, col], n_bins, color = list(mplcyberpunk.cyberpunk_stylesheets['cyberpunk']['axes.prop_cycle'])[2]['color'], marker='.')
    ax3 = add_secondary_plot(train_set, feature, 'predicted_XGB_stroke', ax[row, col], n_bins, color = list(mplcyberpunk.cyberpunk_stylesheets['cyberpunk']['axes.prop_cycle'])[3]['color'], marker='+')
    ax4 = add_secondary_plot(train_set, feature, 'predicted_LGBM_stroke', ax[row, col], n_bins, color = list(mplcyberpunk.cyberpunk_stylesheets['cyberpunk']['axes.prop_cycle'])[4]['color'], marker='x')
    mplcyberpunk.make_lines_glow(ax[row, col])
    
    obj = ax[row, col].get_legend_handles_labels()
    if obj[1][0] not in labels: 
        labels.extend(obj[1])
        handles.extend(obj[0])
    
    obj = ax2.get_legend_handles_labels()

    if obj[1][0] not in labels: 
        labels.extend(obj[1])
        handles.extend(obj[0])
        
    obj = ax3.get_legend_handles_labels()

    if obj[1][0] not in labels: 
        labels.extend(obj[1])
        handles.extend(obj[0])
    
    obj = ax4.get_legend_handles_labels()

    if obj[1][0] not in labels: 
        labels.extend(obj[1])
        handles.extend(obj[0])

fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.04), fontsize=14, ncol=5)

In [ ]:
def cat_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

    param_grid = {
        'depth': trial.suggest_int('depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        #'rsm': trial.suggest_float('rsm', 0.001, 0.9),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100), 
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 20),
        'random_strength': trial.suggest_float('random_strength', 0.001, 0.9),
    } 
    

    clf = CatBoostRegressor(iterations=20000,
                            **param_grid,
                            bootstrap_type='Bernoulli',
                            grow_policy='SymmetricTree',
                            #loss_function='Logloss',
                            eval_metric='AUC',
                            task_type="GPU",
                            random_state=1,)
    
    clf.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)
    preds = clf.predict(X_valid)
    
    #rmse = mean_squared_error(y_val, preds, squared=False)
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    cat_ranks[roc_auc] = clf
    
    # Returns the best RMSE for the trial.
    # Readers may want to try returning a cross validation score here.
    print(roc_auc)
    return roc_auc

In [ ]:
cat_ranks = {}

train_time = 1 * 60 * 60
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='XGBRegressor')
study.optimize(cat_objective, timeout=train_time)



In [ ]:
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('\tValue: {}'.format(trial.value))
print('\tParams: ')
for key, value in trial.params.items():
    print('\t\t{}: {}'.format(key, value))

In [ ]:
trial.params

In [ ]:
len(cat_ranks.keys())

In [ ]:
top_50 = sorted(list(cat_ranks.keys()))[-100:]
top_50

In [ ]:
me_preds = []
for key in top_50:
    me_preds.append(cat_ranks[key].predict(X_test))

final_preds = np.stack(me_preds).mean(0)

In [ ]:
cat_submission = pd.DataFrame(data={'id': test_set.id, 'stroke': final_preds})
cat_submission.to_csv(fr'D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\CAT.csv', index=False)

In [ ]:
me_preds = []
for key in top_50:
    me_preds.append(cat_ranks[key].predict(X))

final_preds = np.stack(me_preds).mean(0)
train_set['predicted_CAT_stroke'] = final_preds

In [ ]:
render_feature_distros(train_df=train_set, test_df=test_set, features=features, labels=['stroke', 'predicted_CAT_stroke'])

In [ ]:
train_set.to_csv('oof_set.csv')

In [ ]:
render_feature_distros(train_df=train_set, test_df=test_set, features=features, labels=['stroke', 'predicted_CAT_stroke'])

In [ ]:
combos = []
for x in range(100):
    for y in range(100):
        if x + y < 100:
            z = 100 - x - y
            combos.append([x, y, z])
combos = np.array(combos) * ([[0.01]*3]*len(combos))
len(combos)

In [ ]:
combos[0:20]

In [ ]:
def roc_auc_score(y_valid, preds):
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)
    return roc_auc

In [ ]:
coefficients = combos
best_score = 0
best_coefficients_index = 0

for index, (a, b, c) in tqdm(enumerate(coefficients)):
    score = roc_auc_score(train_set["stroke"], (a * train_set['predicted_XGB_stroke']) + (b * train_set['predicted_LGBM_stroke']) + (c * train_set['predicted_CAT_stroke']))
    if score > best_score:
        best_score = score
        best_coefficients_index = index

In [ ]:
combos[best_coefficients_index]

In [ ]:
coef = combos[best_coefficients_index]
final_test_preds = XGB_submission['stroke']*coef[0] + LGBM_submission['stroke']*coef[1] + cat_submission['stroke']*coef[2]
final_submission = pd.DataFrame(data={'id': test_set.id, 'stroke': final_test_preds})
final_submission.to_csv(fr'D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\final_submission.csv', index=False)